# What is Experta?

Experta is a python library to build knowledge-based systems.

## The Basics

- Facts
- DefFacts
- Rules
- Knowledge Engine

## Facts

Facts are the basic unit of information of Experta. They are used by the system to reason about the problem.

#### Let's enumerate some facts about Facts:

- The class Fact is a subclass of dict.

In [1]:
from experta import *

In [2]:
f = Fact( a=1, b=2 )
print(f['a'])

1


- Therefore a Fact does not maintain an internal order of items.

In [3]:
f = Fact( a=1, b=2 ) # Order is arbitrary
f = Fact( b=2, a=1 )

- In contrast to dict, you can create a Fact without keys (only values), and Fact will create a numeric index for your values.

In [4]:
f = Fact('x', 'y', 'z')
print(f[0])

x


- You can mix autonumeric values with key-values, but autonumeric must be declared first.

In [5]:
f = Fact('x', 'y', 'z', a=1, b=2)
print(f[0])
print(f['a'])

x
1


- You can subclass Fact to express different kinds of data or extend it with your custom functionality.

In [6]:
class Alert(Fact):
    pass

class Status(Fact):
    pass

f1 = Alert(color = 'red')
f2 = Status(state = 'critical')

print(f1['color'])
print(f2['state'])

red
critical


## DefFacts

- Most of the time expert systems needs a set of facts to be present for the system to work. This is the purpose of the DefFacts decorator.

In [7]:
@DefFacts()
def neededData():
    yield Fact(bestColor = 'red')
    yield Fact(bestBody = 'medium')
    yield Fact(bestSweetness = 'dry')

- All DefFacts inside a knowledgeEngine will be called every time the reset method is called.

## Rules

- In Experta a rule is a callable, decorated with Rule.



- Rules have two component, LHS (left-hand-side) and RHS (right-hand-side) :
    - The LHS describes (using patterns) the conditions on which the rule X should be executed (or fired).
    - The RHS is the set of actions to perform when the rule is fired.
    

- For a Fact to match a Pattern, all pettern restrictions must be True when the fact is evaluated against it.

#### Examples:

- This rule will match with every instance of "MyFact".

In [8]:
class MyFact(Fact):
    pass

@Rule(MyFact()) # This is the LHS
def matchWithEveryMyFact():
    # This is the RHS
    pass

- This rule will match with every instance of "MyFact" wich:
    - f[0] == 'animal'
    - f['family'] == 'felinae'

In [9]:
class MyFact(Fact):
    pass

@Rule(MyFact('animal', family='felinae'))
def matchWithCats():
    print('Meow!')

## KnowledgeEngine

- This is the place where the magic happens.

- The first step is to make a subclass of it and use Rule to decorate its methods.

- After that, you can instantiate it, populate it with facts, and finally run it.


In [10]:
class Greetings(KnowledgeEngine):
    @DefFacts()
    def _init_action(self):
        yield Fact(action = 'greet')
        
    @Rule(Fact(action = 'greet'), NOT(Fact(name = W())))
    def ask_name(self):
        self.declare(Fact(name = input("What's your name? ")))
        
    @Rule(Fact(action = 'greet'),Fact(name = W()), NOT(Fact(location = W())))
    def ask_location(self):
        self.declare(Fact(location = input("Where are your? ")))
        
    @Rule(Fact(action = 'greet'), Fact(name ="a"<< W()), Fact(location ="b"<< W()))
    def greet(self, a, b):
        print("Hi %s! How is the weather in %s?" % (a, b))
        

In [12]:
engine = Greetings()
engine.reset() # Prepare the engine for the execution
engine.run() # Run it

What's your name? ahmad
Where are your? Damascus
Hi ahmad! How is the weather in Damascus?
